In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,42413
2,Huelva,Confirmados PDIA 14 días,2626
3,Huelva,Tasa PDIA 14 días,"511,7212619599743"
4,Huelva,Confirmados PDIA 7 días,994
5,Huelva,Tasa PDIA 7 dias,"193,69799481653254"
6,Huelva,Total Confirmados,42612
7,Huelva,Curados,35819
8,Huelva,Fallecidos,397


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  42413.0


/tmp/ipykernel_26474/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  11836.0


/tmp/ipykernel_26474/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_26474/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_26474/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_26474/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 547 personas en los últimos 7 días 

Un positivo PCR cada 199 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,42413.0,994.0,2626.0,513170.0,193.697995,511.721262,131.0
Huelva-Costa,24934.0,608.0,1561.0,289548.0,209.982455,539.116140,66.0
Condado-Campiña,13331.0,316.0,867.0,156231.0,202.264595,554.947482,52.0
Huelva (capital),11836.0,263.0,723.0,143837.0,182.845860,502.652308,39.0
Sierra de Huelva-Andévalo Central,3754.0,62.0,185.0,67391.0,92.000415,274.517369,13.0
Lepe,2807.0,83.0,206.0,27880.0,297.704448,738.880918,10.0
Rociana del Condado,643.0,14.0,32.0,7939.0,176.344628,403.073435,8.0
Moguer,1844.0,34.0,136.0,21867.0,155.485435,621.941739,8.0
Ayamonte,1838.0,71.0,156.0,21104.0,336.429113,739.196361,6.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Sanlúcar de Guadiana,31.0,3.0,19.0,403.0,744.416873,4714.640199,0.0
Cañaveral de León,18.0,3.0,7.0,396.0,757.575758,1767.676768,0.0
Corteconcepción,43.0,1.0,8.0,536.0,186.567164,1492.537313,0.0
Escacena del Campo,207.0,7.0,31.0,2287.0,306.077831,1355.487538,1.0
Villarrasa,214.0,5.0,26.0,2211.0,226.142017,1175.938489,0.0
Nava (La),7.0,1.0,3.0,258.0,387.596899,1162.790698,0.0
Campofrío,18.0,2.0,7.0,713.0,280.504909,981.767181,0.0
Paterna del Campo,188.0,12.0,33.0,3457.0,347.121782,954.584900,5.0
Trigueros,516.0,35.0,72.0,7862.0,445.179344,915.797507,5.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Minas de Riotinto,166.0,1.0,11.0,3812.0,26.232949,288.562434,0.0,0.090909
Corteconcepción,43.0,1.0,8.0,536.0,186.567164,1492.537313,0.0,0.125000
Nerva,242.0,1.0,7.0,5169.0,19.346102,135.422712,0.0,0.142857
Sanlúcar de Guadiana,31.0,3.0,19.0,403.0,744.416873,4714.640199,0.0,0.157895
Valverde del Camino,798.0,5.0,27.0,12750.0,39.215686,211.764706,0.0,0.185185
Villarrasa,214.0,5.0,26.0,2211.0,226.142017,1175.938489,0.0,0.192308
Alosno,221.0,1.0,5.0,3933.0,25.425884,127.129418,0.0,0.200000
Paymogo,85.0,1.0,5.0,1162.0,86.058520,430.292599,0.0,0.200000
Calañas,240.0,1.0,5.0,2768.0,36.127168,180.635838,0.0,0.200000
